# __Prueba - SQL para Data Science__

Desde __OkCupid__ -aplicación de citas- solicitan el desarrollo de una serie de modelos
predictivos.

Los datos a utilizar se registraron en base a una serie de perfiles públicos dentro de 25
millas de la ciudad de San Francisco activos durante el 2011.

__*Caveat*__: Los permisos para obtener estos datos provinieron del presidente y co-fundador de
OkCupid, Christian Rudder, con la condición que se mantuvieran públicos.


## Requerimientos

### Parte 1: Registro de los archivos en la base de datos.

- Generar una nueva base de datos con la siguiente nomenclatura: apellido_nombre.
- Importar en tablas los archivos `train_cupid.csv` y `test_cupid.csv` a un motor
Postgres, __implementando sólo la librería `psycopg2`__. Las tablas deben contener los
nombres de las columnas y el total de los registros presente en cada archivo.

In [226]:
import psycopg2
import pandas as pd
import numpy as np
# Creamos la conexión a la base de datos
conn = psycopg2.connect("user=postgres password=******")
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

In [227]:
# Creamos la base de datos
cur = conn.cursor()
cur.execute("CREATE DATABASE benjamin_meneses")
conn.commit()
# Cerramos y nos conectamos a la base de datos creada
cur.close()
conn.close()
conn = psycopg2.connect("dbname=benjamin_meneses user=postgres password=******")

In [228]:
# Arreglo con nombre de las columnas
csvHeader = "age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,aquarius,cancer,sagittarius,asian,hispanic / latin,black,indian,pacific islander,native american,middle eastern,colorado,new york,oregon,arizona,hawaii,montana,wisconsin,virginia,spain,nevada,illinois,vietnam,ireland,louisiana,michigan,texas,united kingdom,massachusetts,north carolina,idaho,mississippi,new jersey,florida,minnesota,georgia,utah,washington,west virginia,connecticut,tennessee,rhode island,district of columbia,canada,missouri,germany,pennsylvania,netherlands,switzerland,mexico,ohio,agnosticism,atheism,catholicism,buddhism,judaism,hinduism,islam,pro_dogs,pro_cats,spanish,chinese,french,german,single,seeing_someone,available,employed,income_between_25_50,income_between_50_75,income_over_75,drugs_often,drugs_sometimes,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,orientation_gay,orientation_straight,sex_m,smokes_sometimes,smokes_trying to quit,smokes_when drinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university"
tableCols = list(map(lambda x: x.replace(' ', '_').replace('/', ''), csvHeader.split(',')))
# Construimos las tablas
trainSql = "CREATE TABLE train (id SERIAL PRIMARY KEY"
testSql = "CREATE TABLE test (id SERIAL PRIMARY KEY"
for i, col in enumerate(tableCols):
    trainSql += ", "
    testSql += ", "
    if i > 1:
        trainSql += col + " BOOLEAN NOT NULL"
        testSql += col + " BOOLEAN NOT NULL"
    else:
        trainSql += col + " integer NOT NULL"
        testSql += col + " integer NOT NULL"
trainSql += ");"
testSql += ");"

In [229]:
# Creamos el cursor
cur = conn.cursor()
# Creamos la tabla train y conservamos los cambios
cur.execute(trainSql)
conn.commit()
# Creamos la tabla test y conservamos los cambios
cur.execute(testSql)
conn.commit()
# Cerramos el cursor
cur.close()

In [230]:
# Importamos los csv para poblar las tablas
trainDf = pd.read_csv('train_cupid.csv')
testDf = pd.read_csv('test_cupid.csv')
# Recodificamos los booleanos
trainDf[trainDf.columns[2:]] = trainDf[trainDf.columns[2:]].astype(bool)
testDf[testDf.columns[2:]] = testDf[testDf.columns[2:]].astype(bool)

In [231]:
cur = conn.cursor()
try:
    # Insertamos los valores SQL
    for index, row in trainDf.iterrows():
        rowQuery = f"INSERT INTO train ({', '.join(tableCols)}) VALUES ("
        rowQuery +=f"{', '.join(['%s' for i in tableCols])});\n"
        cur.execute(rowQuery, [str(val) if type(val) == 'bool' else val for val in row.values])
    for index, row in testDf.iterrows():
        rowQuery = f"INSERT INTO test ({', '.join(tableCols)}) VALUES ("
        rowQuery +=f"{', '.join(['%s' for i in tableCols])});\n"
        cur.execute(rowQuery, [str(val) if type(val) == 'bool' else val for val in row.values])
    conn.commit()
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: unable to insert data")
    print(error)
    conn.rollback()
    cur.close()


### Parte 2: Entrenamiento de modelos (3.5 Puntos)
- Ingestar la tabla de training __mediante__ `psycopg2` para el posterior entrenamiento del
modelo.
- Entrenar los siguientes modelos (sin necesidad de ajustar por hiper parámetros):
   - `GradientBoostingClassifier`, `AdaBoostClassifer`,
    `RandomForestClassifier`, `SVC`, `DecisionTreeClassifier`,
    `LogisticRegression`, `BernoulliNB`.
   - Existen tres vectores objetivos a evaluar: single, seeing someone y available.
- Serializar el objeto y preservarlo por cada combinación de modelo entrenado y vector
objetivo.


In [232]:
# importamos los modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB


In [233]:
# Conseguimos la data de entrenamiento
cur = conn.cursor()
trainData = []
try:
    cur.execute("SELECT * FROM train;")
    trainData = cur.fetchall()
    trainData = pd.DataFrame(trainData, columns=['id', *tableCols])
    trainData = pd.concat([trainData.loc[:, ['age', 'height']], pd.get_dummies(trainData.loc[:, 'virgo':]).astype(np.int8)], axis=1)
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: unable to fetch data")
    print(error)
    conn.rollback()
    cur.close()

In [234]:
# Entrenamos los modelos para cada vector objetivo
targets = ['single', 'seeing_someone', 'available']
modelsHash = {}
X_train = trainData.drop(columns=targets)
for col in targets:
    modelsHash[col] = {}
    y_train = trainData.loc[:, col]
    modelsHash[col]['DecisionTree'] = DecisionTreeClassifier(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['RandomForest'] = RandomForestClassifier(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['AdaBoost'] = AdaBoostClassifier(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['GradientBoosting'] = GradientBoostingClassifier(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['SVC'] = SVC(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['LogisticRegression'] = LogisticRegression(random_state=19137).fit(X_train, y_train)
    modelsHash[col]['BernoulliNB'] = BernoulliNB().fit(X_train, y_train)


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

### Parte 3: Exportación de predicciones (3.5 Puntos)
- Ingestar la tabla de testing __mediante__ psycopg2 para la posterior predicción del
modelo.
- __En base a los objetos serializados__, predecir y evaluar cuatro queries específicas:
   - __Query 1:__ 'atheism', 'asian', 'employed', 'pro_dogs', 'chinese'.
   - __Query 2:__ 'income_over_75', 'french', 'german','orientation_straight', 'new york'.
   - __Query 3:__ 'education_undergrad_university', 'body_type_regular', 'pro_dogs',
'employed'.
   - __Query 4:__ 'taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism'.
- Cada una de estas queries específicas debe ser registrada en la base de datos.
- La base de datos creada debe contener las tablas:
   - 2 que representan a training y testing.
   - 84 que representan a cada una de las combinaciones entre modelo, vector y
query específica.
- A modo de referencia, la base de datos creada debe contener 86 tablas en total.


In [235]:
queries = [
    "WHERE atheism AND asian AND employed AND pro_dogs AND chinese",
    "WHERE income_over_75 AND french AND german AND orientation_straight AND new_york",
    "WHERE education_undergrad_university AND body_type_regular AND pro_dogs AND employed",
    "WHERE taurus AND indian AND washington AND income_between_50_75 AND hinduism"
]

cur = conn.cursor()
try:
    for number, query in enumerate(queries):
        print("="*20)
        print(f"Query {number+1}: {query}")
        cur.execute(f"SELECT * FROM test {query};")
        data = cur.fetchall()
        data = pd.DataFrame(data, columns=['id', *tableCols])
        if len(data) == 0:
            for col in targets:
                for model in modelsHash[col]:
                    cur.execute(f"CREATE TABLE query{number + 1}_{col}_{model.lower()} (id_obs integer PRIMARY KEY, prediction BOOLEAN NOT NULL, FOREIGN KEY (id_obs) REFERENCES test (id));")
                    conn.commit()
            continue
        ids = data['id']
        data = pd.concat([data.loc[:, ['age', 'height']], pd.get_dummies(data.loc[:, 'virgo':]).astype(np.int8)], axis=1)
        X_test = data.drop(columns=targets)
        for col in targets:
            y_test = data.loc[:, col]
            print(f"{col}")
            for model in modelsHash[col]:
                print(f"\t{model}")
                print(f"\t\t{modelsHash[col][model].score(X_test, y_test)}")
                try:
                    cur.execute(f"CREATE TABLE query{number + 1}_{col}_{model.lower()} (id_obs integer PRIMARY KEY, prediction BOOLEAN NOT NULL, FOREIGN KEY (id_obs) REFERENCES test (id));")
                    conn.commit()
                    for idx, id in ids.items():
                        prediction = modelsHash[col][model].predict([X_test.iloc[idx]])[0]
                        cur.execute(f"INSERT INTO query{number + 1}_{col}_{model.lower()} (id_obs, prediction) VALUES (%s, %s);", [id, bool(prediction)])
                    conn.commit()
                except (Exception, psycopg2.DatabaseError) as error:
                    conn.rollback()
                    print(f"Error: unable to create table query{number + 1}_{col}_{model}")
                    print(error)
    conn.commit()
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: unable to fetch data")
    print(error)
    conn.rollback()
    cur.close()

Query 1: WHERE atheism AND asian AND employed AND pro_dogs AND chinese
single
	DecisionTree
		0.8333333333333334
	RandomForest
		0.8666666666666667
	AdaBoost
		0.8666666666666667
	GradientBoosting
		0.8666666666666667
	SVC
		0.8666666666666667
	LogisticRegression
		0.8666666666666667
	BernoulliNB
		0.9
seeing_someone
	DecisionTree
		0.8333333333333334
	RandomForest
		0.9
	AdaBoost
		0.9
	GradientBoosting
		0.9
	SVC
		0.9
	LogisticRegression
		0.9
	BernoulliNB
		0.9
available
	DecisionTree
		0.8333333333333334
	RandomForest
		0.9
	AdaBoost
		0.9
	GradientBoosting
		0.9
	SVC
		0.9
	LogisticRegression
		0.9
	BernoulliNB
		0.9
Query 2: WHERE income_over_75 AND french AND german AND orientation_straight AND new_york
Query 3: WHERE education_undergrad_university AND body_type_regular AND pro_dogs AND employed
single
	DecisionTree
		0.8931383577052868
	RandomForest
		0.9516310461192351
	AdaBoost
		0.952755905511811
	GradientBoosting
		0.952755905511811
	SVC
		0.952755905511811
	LogisticRegres

In [236]:
conn.close()